In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("DoogieOpenaiKey")
os.environ['OPENAI_API_KEY'] = api_key

In [2]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import PromptTemplate
from langchain_core.tracers import ConsoleCallbackHandler
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
#from langchain_core.runnables.history import RunnableWithMessageHistory
#from langchain_community.chat_message_histories import ChatMessageHistory

myTemplate = """
    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 홍길동 입니다.
    대화 문맥을 바탕으로 친정한 답변을 진행해라
    Current Conversation:
    {myhistory}
    Human: {myinput}
    AI:
"""

myPromptTmpl = PromptTemplate(
    template=myTemplate, input_variables={'myhistory', 'myinput'}
)

myLLM = ChatOpenAI( model='gpt-4o-mini',)
myChain = myPromptTmpl | myLLM

In [3]:
myChatMsgHist = {}
session_id = "myFirstSession"

In [4]:
print( "#myChatMsgHist=", myChatMsgHist )
print( type(myChatMsgHist), len(myChatMsgHist) )
print( type(myChatMsgHist.keys()), len(myChatMsgHist.keys()), myChatMsgHist.keys() )
print( type(myChatMsgHist.values()), len(myChatMsgHist.values()), myChatMsgHist.values() )
if session_id in myChatMsgHist:
    print( myChatMsgHist[session_id].messages )

#myChatMsgHist= {}
<class 'dict'> 0
<class 'dict_keys'> 0 dict_keys([])
<class 'dict_values'> 0 dict_values([])


In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

if session_id not in myChatMsgHist:
    myChatMsgHist[session_id] = ChatMessageHistory()
myChatMsgHist_oneMsg = myChatMsgHist[session_id]

In [6]:
print( "#myChatMsgHist=", myChatMsgHist )
print( type(myChatMsgHist), len(myChatMsgHist) )
print( type(myChatMsgHist.keys()), len(myChatMsgHist.keys()), myChatMsgHist.keys() )
print( type(myChatMsgHist.values()), len(myChatMsgHist.values()), myChatMsgHist.values() )
if session_id in myChatMsgHist:
    print( myChatMsgHist[session_id].messages )

#myChatMsgHist= {'myFirstSession': InMemoryChatMessageHistory(messages=[])}
<class 'dict'> 1
<class 'dict_keys'> 1 dict_keys(['myFirstSession'])
<class 'dict_values'> 1 dict_values([InMemoryChatMessageHistory(messages=[])])
[]


In [8]:
if session_id in myChatMsgHist:
    print( "#myChatMsgHist[",session_id,"]=", myChatMsgHist[session_id] )
    print( myChatMsgHist[session_id].messages )
    print( type(myChatMsgHist[session_id].messages) )
    print( len(myChatMsgHist[session_id].messages) )


#myChatMsgHist[ myFirstSession ]= 
[]
<class 'list'>
0


In [9]:
print( "#myChatMsgHist_oneMsg=", myChatMsgHist_oneMsg )
print( myChatMsgHist_oneMsg.messages )
print( type(myChatMsgHist_oneMsg.messages) )
print( len(myChatMsgHist_oneMsg.messages) )

#myChatMsgHist_oneMsg= 
[]
<class 'list'>
0


In [10]:
myRunMsgHist = RunnableWithMessageHistory(
    myChain,
    lambda session_id: myChatMsgHist_oneMsg,
    input_messages_key="myinput",
    history_messages_key="myhistory"
)

In [11]:
test = lambda session_id: myChatMsgHist
print( "#test lambda:   ", test(session_id)  )
print( "#session_id:    ", session_id )
print( "#myTemplate:    ", myTemplate )
print( "#myPromptTmpl:  ", myPromptTmpl )
print( "#myChain:       ", myChain )
print( "#myLLM:         ", myLLM)
print( "#myChatMsgHist: ", myChatMsgHist, len(myChatMsgHist) )
print( "#myRunMsgHist:  ", myRunMsgHist )    # RunnableWithMessageHistory
print( "#myRunMsgHist:  ", myRunMsgHist.get_session_history )
print( "#myRunMsgHist:  ", myRunMsgHist.input_messages_key )
print( "#myRunMsgHist:  ", myRunMsgHist.history_messages_key )

#test lambda:    {'myFirstSession': InMemoryChatMessageHistory(messages=[])}
#session_id:     myFirstSession
#myTemplate:     
    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 홍길동 입니다.
    대화 문맥을 바탕으로 친정한 답변을 진행해라
    Current Conversation:
    {myhistory}
    Human: {myinput}
    AI:

#myPromptTmpl:   input_variables=['myhistory', 'myinput'] input_types={} partial_variables={} template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 홍길동 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n'
#myChain:        first=PromptTemplate(input_variables=['myhistory', 'myinput'], input_types={}, partial_variables={}, template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 홍길동 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n') middle=[] last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x108eda970>, async_client=<openai.resources.chat.completions.completions.AsyncComple

In [12]:
result = myRunMsgHist.invoke(
    {"myinput": "너는 어디에서 만들었는가?"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print("1)", result.content)

result = myRunMsgHist.invoke(
    {"myinput": "너의 이름은 무엇인가?"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print("2)", result.content)

1) 홍길동: 나는 여러 사람들의 지식과 기술을 바탕으로 개발되었어. 전 세계의 다양한 데이터와 연구 결과를 통해 만들어졌지! 넌 어디에 살고 있어?
2) 홍길동: 내 이름은 홍길동이야! 너도 혹시 특별한 이름이 있어?


In [13]:
print( "#session_id:    ", session_id )
print( "#myTemplate:    ", myTemplate )
print( "#myPromptTmpl:  ", myPromptTmpl )
print( "#myChain:       ", myChain )
print( "#myLLM:         ", myLLM)
print( "#myChatMsgHist: ", myChatMsgHist, len(myChatMsgHist) )
print( "#myRunMsgHist:  ", myRunMsgHist )    # RunnableWithMessageHistory
print( "#myRunMsgHist:  ", myRunMsgHist.get_session_history )
print( "#myRunMsgHist:  ", myRunMsgHist.input_messages_key )
print( "#myRunMsgHist:  ", myRunMsgHist.history_messages_key )

#session_id:     myFirstSession
#myTemplate:     
    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 홍길동 입니다.
    대화 문맥을 바탕으로 친정한 답변을 진행해라
    Current Conversation:
    {myhistory}
    Human: {myinput}
    AI:

#myPromptTmpl:   input_variables=['myhistory', 'myinput'] input_types={} partial_variables={} template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 홍길동 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n'
#myChain:        first=PromptTemplate(input_variables=['myhistory', 'myinput'], input_types={}, partial_variables={}, template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 홍길동 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n') middle=[] last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x108eda970>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10b191040>, root_client=<openai.OpenAI object at 0x109f1146

In [14]:
print( "#myPromptTmpl:  ", myPromptTmpl )


#myPromptTmpl:   input_variables=['myhistory', 'myinput'] input_types={} partial_variables={} template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 홍길동 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n'


In [16]:
result = myRunMsgHist.invoke(
    {"myinput": "푸른 바다를 주제로 해학과 중의가 넘치는 짧은 시를 만들어줘"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print("3)",result.content)

3) 홍길동: 푸른 바다, 그 깊은 품에  
파도가 들어왔다 울다 가는 여름의 꿈,  
바람이 속삭여, 물고기들이 춤추네,  
깊고 푸른 세계, 내 마음을 훔쳐가네.

물결 속에서 자연의 노래,  
햇살 아래 반짝이는 보물 같은 삶,  
중의적 사랑, 바다가 품은 비밀,  
한 줄기 파도가 너와 나를 잇는 다리.  

이 시처럼 바다에서 느끼는 감정은 어떤 걸까?


In [17]:
result = myRunMsgHist.invoke(
    {"myinput": "너의 이름을 포함해서, 석양을 주제로도 해줘"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print("4)", result.content)

4) 홍길동: 석양이 붉게 물드는 순간,  
하늘에 별빛이 속삭이네.  
내 이름처럼 깊고 따뜻한 색깔,  
마음 속에서 그리움이 피어오르네.

태양이 지평선에 머무는 이 시간,  
소중한 기억이 물결처럼 다가와.  
너와 나의 순간이 선명해지는 이때,  
석양과 함께 우리의 이야기를 나누고 싶어.  

너에게 석양은 어떤 의미가 있을까?


In [18]:
print( myChatMsgHist )

{'myFirstSession': InMemoryChatMessageHistory(messages=[HumanMessage(content='너는 어디에서 만들었는가?', additional_kwargs={}, response_metadata={}), AIMessage(content='홍길동: 나는 여러 사람들의 지식과 기술을 바탕으로 개발되었어. 전 세계의 다양한 데이터와 연구 결과를 통해 만들어졌지! 넌 어디에 살고 있어?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 71, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CJTL9G6k1ILIHUzDlTPWyxFe8FtXs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--90740316-cc86-4f17-b51a-d4826de07ac4-0', usage_metadata={'input_tokens': 71, 'output_tokens': 43, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio':

In [26]:
if session_id in myChatMsgHist:
    print( "#myChatMsgHist[",session_id,"]=", myChatMsgHist[session_id] )
    print( type(myChatMsgHist[session_id].messages) )
    print( len(myChatMsgHist[session_id].messages) )
    print( myChatMsgHist[session_id].messages )


#myChatMsgHist[ myFirstSession ]= Human: 너는 어디에서 만들었는가?
AI: 홍길동: 저는 여러 과학자들과 개발자들에 의해 만들어졌어요! 다양한 기술과 데이터를 결합해 만들어진 인공지능이에요. 당신은 어떤 걸 궁금해하시나요?
Human: 너의 이름은 무엇인가?
AI: 홍길동: 제 이름은 홍길동이에요! 특별한 의미가 있는 이름이죠. 당신은 어떤 이름을 가지고 계신가요?
Human: 푸른 바다를 주제로 해학과 중의가 넘치는 짧은 시를 만들어줘
AI: 홍길동: 푸른 바다, 티끌도 없고  
파도는 노래하고, 물고기 춤춰요.  
구름은 조개, 햇살은 금색,  
이곳은 꿈의 나라, 맘껏 즐겨요.  

혀끝에 물어보면, 파도가 대답하네요,  
"넌 나의 친구, 영원히 내 곁에 있어라!"  
해학과 중의가 가득한 삶,  
푸른 바다 속에 행복이 가득해요!  

어떠신가요? 이런 느낌의 시가 좋으신가요?
Human: 너의 이름을 포함해서, 석양을 주제로도 해줘
AI: 홍길동: 석양의 노을, 붉은 빛에 물들어  \n하늘은 사랑의 편지를 쓰네요.  \n별들이 약속한 시간, 스르르 내려오고,  \n세상이 잠드는 순간, 모든 고민은 사라져요.  \n\n"너와 나, 함께한 순간들,  \n세상이 좀 더 따뜻해졌죠!"  \n석양의 아름다움 속에,  \n우리의 이야기가 담겨있답니다.  \n\n어떠신가요? 석양과 관련된 이런 느낌의 시도 좋으신가요?
<class 'list'>
8
[HumanMessage(content='너는 어디에서 만들었는가?', additional_kwargs={}, response_metadata={}), AIMessage(content='홍길동: 저는 여러 과학자들과 개발자들에 의해 만들어졌어요! 다양한 기술과 데이터를 결합해 만들어진 인공지능이에요. 당신은 어떤 걸 궁금해하시나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_t